In [1]:
!python --version

Python 3.8.0


In [2]:
# !poetry remove pandas

In [3]:

# !poetry add tensorflow=2.5 tensorflow-text=2.5 tensorflow-hub 
# numpy=1.19.3 
# pandas==1.2.5 openpyxl

!poetry add pandas==1.2.5 openpyxl

^C


In [1]:
import tensorflow_hub as hub
import numpy as np
import tensorflow_text

from scipy import spatial

In [2]:
# Some texts of different lengths.
english_sentences = ["dog", "Puppies are nice.", "I enjoy taking long walks along the beach with my dog."]
italian_sentences = ["cane", "I cuccioli sono carini.", "Mi piace fare lunghe passeggiate lungo la spiaggia con il mio cane."]
japanese_sentences = ["犬", "子犬はいいです", "私は犬と一緒にビーチを散歩するのが好きです"]

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

In [3]:
# Compute embeddings.
en_result = embed(english_sentences)
it_result = embed(italian_sentences)
ja_result = embed(japanese_sentences)

In [4]:
en_result.numpy().shape

(3, 512)

In [5]:
# Compute similarity matrix. Higher score indicates greater similarity.
similarity_matrix_it = np.inner(en_result, it_result)
similarity_matrix_ja = np.inner(en_result, ja_result)

In [6]:
similarity_matrix_it

array([[0.957878  , 0.3308628 , 0.30247933],
       [0.38761458, 0.73392963, 0.2481403 ],
       [0.23617546, 0.21800505, 0.92830104]], dtype=float32)

In [7]:
import pandas as pd

In [8]:
df = pd.read_excel("ЭТ_Новости.xlsx")
df.head()

,ID,NEWS_HEAD
0,1552621575,Государственная Дума рекомендует Правительству...
1,1552115547,Евгений Грабчак: «Меры поддержки помогут ТЭК о...
2,1552115549,Павел Сниккарс: «Важная задача в электроэнерге...
3,1551704812,Вниманию гарантирующих поставщиков и энергосбы...
4,1551704814,Эдуард Шереметцев: «Национальная энергетическа...


In [9]:
%%time
df["emb"] = df["NEWS_HEAD"].apply(lambda x: embed(x).numpy()[0])

Wall time: 10.2 s


In [10]:
df.head()

,ID,NEWS_HEAD,emb
0,1552621575,Государственная Дума рекомендует Правительству...,"[6.6673114e-05, 0.02740392, -0.033854015, -0.0..."
1,1552115547,Евгений Грабчак: «Меры поддержки помогут ТЭК о...,"[0.014879701, -0.0015657819, 0.031823713, -0.0..."
2,1552115549,Павел Сниккарс: «Важная задача в электроэнерге...,"[-0.024322681, 0.063832596, 0.006030567, -0.04..."
3,1551704812,Вниманию гарантирующих поставщиков и энергосбы...,"[0.029627595, 0.06477559, -0.015955612, 0.0087..."
4,1551704814,Эдуард Шереметцев: «Национальная энергетическа...,"[-0.028674196, -0.037707165, -0.0035214513, -0..."


In [11]:
df.to_pickle("../News+emb.pkl")

In [12]:
emb_ar = np.stack(df.emb.to_numpy())
emb_ar.shape

(1975, 512)

In [13]:
def get_closest_to_text(text, df, topN=5):
    print(f"Input string:\n{text}\n")
    emb_ar = np.stack(df.emb.to_numpy())
    text_emb = embed(text)
    cos_dist = np.array([spatial.distance.cosine(text_emb, x) for x in emb_ar])
    
    print("Closest records:")
    top_args = np.argpartition(cos_dist, topN)[:topN]
    n_df = df.copy().iloc[top_args,:]
    n_df["score"] = cos_dist[top_args]
    return n_df.sort_values("score", ascending=True)

In [14]:
get_closest_to_text(df.iloc[0].NEWS_HEAD, df, topN=5)

Input string:
Государственная Дума рекомендует Правительству РФ, несмотря на кризис, вызванный санкциями недружественных стран, сохранить рыночное ценообразование на нефтепродукты и электроэнергию на внутреннем рынке и продолжить разработку мер поддержки новых отраслей ТЭК, связанных с переходом на низкоуглеродную траекторию развития экономики.

Closest records:


,ID,NEWS_HEAD,emb,score
0,1552621575,Государственная Дума рекомендует Правительству...,"[6.6673114e-05, 0.02740392, -0.033854015, -0.0...",0.000000
1742,1100853539,Комитет Государственной Думы по энергетике про...,"[-0.026314128, 0.07819955, -0.043848775, -0.01...",0.572938
1735,1100853537,Комитет Государственной Думы по энергетике про...,"[-0.01887728, 0.07468432, -0.044374257, -0.024...",0.607846
531,1255968164,Павел Завальный: «Намерение ЕС форсировать эне...,"[-0.005707043, -0.08757431, -0.029431945, -0.0...",0.610513
478,1353506324,Депутаты комитета Государственной Думы по энер...,"[-0.025714297, 0.02512893, -0.039344855, -0.05...",0.614853


In [15]:
get_closest_to_text(df.iloc[2].NEWS_HEAD, df, topN=10)

Input string:
Павел Сниккарс: «Важная задача в электроэнергетике в период высокой турбулентности – сохранить отлаженные за 10 лет механизмы»

Closest records:


,ID,NEWS_HEAD,emb,score
2,1552115549,Павел Сниккарс: «Важная задача в электроэнерге...,"[-0.024322681, 0.063832596, 0.006030567, -0.04...",0.000000
1230,1100852945,Александр Новак: «Конкурс «Лидеры энергетики» ...,"[-0.020745384, 0.020943524, 0.052267842, -0.01...",0.532184
1016,1110168307,Евгений Грабчак: «Цифровая модернизация позвол...,"[-0.028432861, 0.03438398, -0.009605768, -0.01...",0.546007
1716,1100853535,Комитет Государственной Думы по энергетике про...,"[0.0143666975, 0.065282606, -0.0020122027, -0....",0.557654
1192,1100853113,Павел Сорокин: «РЭН-2019 соберет экспертов в н...,"[0.009449594, 0.0034494305, -0.029740738, -0.0...",0.563434
1500,1100853214,Евгений Грабчак: «Цифровые технологии позволят...,"[0.008983374, 0.060951285, 0.012339252, -0.008...",0.565334
1194,1100853117,Павел Сниккарс провел заседание по актуализаци...,"[0.037695155, 0.041363195, 0.028168043, -0.045...",0.566679
55,1532569895,Евгений Грабчак: «Долгосрочное планирование в ...,"[-0.031916738, 0.04177955, -0.05964597, -0.082...",0.572095
898,1157826551,Комитет Государственной Думы по энергетике про...,"[0.048303206, 0.0815241, 0.0081711095, -0.0134...",0.575072
1112,1100852913,Александр Новак: «Уровень запасов топлива к зи...,"[-0.0022129382, 0.056396365, 0.037317578, -0.0...",0.579249


In [16]:
get_closest_to_text(df.iloc[18].NEWS_HEAD, df, topN=5)

Input string:
ФАС определила задачи совершенствования тарифного регулирования

Closest records:


,ID,NEWS_HEAD,emb,score
18,1546783300,ФАС определила задачи совершенствования тарифн...,"[0.09777064, 0.026426591, 0.051717404, -0.0283...",0.000000
19,1546783302,ФАС определила задачи совершенствования тарифн...,"[0.09777064, 0.026426591, 0.051717404, -0.0283...",0.000000
201,1477314954,Руководитель ФАС представил основные итоги и п...,"[0.072988935, -0.0012470167, -0.040068354, 0.0...",0.469435
728,1210095965,Цифровизация тарифного регулирования неизбежна,"[0.101398155, 0.0032326966, -0.00889873, 0.037...",0.478472
226,1470097088,"В ДОП приняты изменения, связанные с уточнение...","[0.048701223, -0.021925649, -0.02592163, 0.028...",0.494197


In [17]:
get_closest_to_text(df.iloc[19].NEWS_HEAD, df)

Input string:
ФАС определила задачи совершенствования тарифного регулирования

Closest records:


,ID,NEWS_HEAD,emb,score
18,1546783300,ФАС определила задачи совершенствования тарифн...,"[0.09777064, 0.026426591, 0.051717404, -0.0283...",0.000000
19,1546783302,ФАС определила задачи совершенствования тарифн...,"[0.09777064, 0.026426591, 0.051717404, -0.0283...",0.000000
201,1477314954,Руководитель ФАС представил основные итоги и п...,"[0.072988935, -0.0012470167, -0.040068354, 0.0...",0.469435
728,1210095965,Цифровизация тарифного регулирования неизбежна,"[0.101398155, 0.0032326966, -0.00889873, 0.037...",0.478472
226,1470097088,"В ДОП приняты изменения, связанные с уточнение...","[0.048701223, -0.021925649, -0.02592163, 0.028...",0.494197


In [18]:
get_closest_to_text(df.iloc[142].NEWS_HEAD, df)

Input string:
Уведомление для ООО "Грин Энерджи Рус"! Информация о максимально допустимых величинах потребления на собственные нужды электростанций на 2021 год!

Closest records:


,ID,NEWS_HEAD,emb,score
142,1497423434,"Уведомление для ООО ""Грин Энерджи Рус""! Информ...","[-0.006868089, -0.011465344, 0.038518388, -0.0...",0.000000
450,1353505575,"Уведомление для ООО ""Санлайт Энерджи""! Информа...","[-0.018210936, -0.030568626, 0.050093822, 0.00...",0.044107
510,1353830748,"Уведомление для ООО ""Грин Энерджи Рус"" и АО ""В...","[-0.0045551276, -0.029549306, 0.03328291, -0.0...",0.050182
643,1227444065,"Уведомление для ООО ""Санлайт Энерджи""! Информа...","[-0.015239214, -0.020699851, 0.04711477, 0.001...",0.070868
666,1221731526,"Уведомление для ООО ""Санлайт Энерджи""! Информа...","[-0.015239214, -0.020699851, 0.04711477, 0.001...",0.070868


In [19]:
get_closest_to_text("I love pizza", df)

Input string:
I love pizza

Closest records:


,ID,NEWS_HEAD,emb,score
944,1138231366,С Днем энергетика!,"[0.03395263, 0.027581275, -0.021881483, 0.0482...",0.817647
124,1506307579,С Днем энергетика!,"[0.03395263, 0.027581275, -0.021881483, 0.0482...",0.817647
945,1138208208,С Днем Энергетика!,"[0.045147557, 0.014729199, -0.019837264, 0.042...",0.826592
1574,1100852043,В состав Ассоциации «НП Совет рынка» приняты т...,"[0.012550067, -0.0349957, -0.02011785, -0.0576...",0.826614
265,1404837295,В Члены Ассоциации «НП Совет рынка» приняты тр...,"[0.01721313, -0.028385356, -0.037241697, -0.04...",0.833423
